<a href="https://colab.research.google.com/github/FelipeAugus/dio-baires-dev--machine-learning-training-ipynb/blob/main/01-programacao-para-ml/transfer-leanring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import random
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# Preparo das Imagens

In [2]:
!wget --no-check-certificate https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip -O /tmp/cats-and-dogs.zip
with zipfile.ZipFile('/tmp/cats-and-dogs.zip', 'r') as zip: zip.extractall('/tmp')

--2025-07-17 01:42:33--  https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.222.110.64, 2600:1413:a000:1383::317f, 2600:1413:a000:138c::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.222.110.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M   195MB/s    in 4.4s    

2025-07-17 01:42:37 (180 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



In [5]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

import shutil

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE, MAX_SIZE, CLASS_DIR):
    files = []
    current_size = 0
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0 and current_size<=MAX_SIZE:
            files.append(filename)
            current_size+=1
    print(f'Arquivos separados: {len(files)}')

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + CLASS_DIR + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + CLASS_DIR + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
TESTING_DIR = "/tmp/cats-v-dogs/testing/"

max_size = 120
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_DIR, TESTING_DIR, split_size, max_size, "cats/")
split_data(DOG_SOURCE_DIR, TRAINING_DIR, TESTING_DIR, split_size, max_size, "dogs/")

print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

Arquivos separados: 121
Arquivos separados: 121
121
119
26
24


In [6]:
batch_size = 32

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   height_shift_range=0.3,
                                   width_shift_range=0.3,
                                   rotation_range=50,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/tmp/cats-v-dogs/training',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/tmp/cats-v-dogs/testing',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 456 images belonging to 4 classes.
Found 76 images belonging to 4 classes.


# Instanciando a rede

In [ ]:
model=MobileNet(weights='imagenet', include_top=False)
# Criando a saída do modelo MobileNet:
x=model.output
x=GlobalAveragePooling2D()(x)

# Adicionando uma camada intermediária e a camada final:
x=Dense(50,activation='relu')(x)
preds=Dense(1,activation='sigmoid')(x)
model=Model(inputs=model.input, outputs=preds)

# Visualizando as camadas da nova rede criada usando o modelo MobileNetV2:
# for i,layer in enumerate(model.layers): print(i,layer.name)

# Definindo qual camada da rede será treinada. Nesse caso somente as duas últimas camadas adicionadas:
for layer in model.layers[:86]: layer.trainable=False
for layer in model.layers[86:]: layer.trainable=True

# Treinamento

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

history = model.fit(training_set,
                   steps_per_epoch=int(8000/batch_size),
                   epochs=10,
                   validation_data = test_set,
                   validation_steps = int(2000/batch_size))

In [ ]:
model.save('transferlearning_catsanddogs.h5')
from google.colab import files
files.download('transferlearning_catsanddogs.h5')

# Predição

In [ ]:
test_image = image.load_img('catsxdogs/single_prediction/chino1.jpg', target_size = (224, 224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

In [ ]:
result = model.predict(test_image)

if result[0][0] > 0.5: prediction = 'dog'
else: prediction = 'cat'

print(prediction)